# Serverless

- Name: Aryan Ebrahimpour
- ID: 98726942

In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.14"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.14"
#r "nuget: FSharp.Data"
#r "nuget: NodaTime"
#r "nuget: MathNet.Numerics"
#r "nuget: MathNet.Numerics.FSharp"

open System
open Plotly.NET
open Plotly.NET.Interactive
open NodaTime
open System.Text.Json
open System.Text.Json.Serialization

Installed Packages FSharp.Data, 4.2.7 MathNet.Numerics, 4.15.0 MathNet.Numerics.FSharp, 4.15.0 NodaTime, 3.0.9 Plotly.NET, 2.0.0-preview.14 Plotly.NET.Interactive, 2.0.0-preview.14

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
let (=>) a b = a, b
let mapOf = Map.ofList

let mapKeys m = m |> Map.toSeq |> Seq.map fst
let mapValues m = m |> Map.toSeq |> Seq.map snd

In [ ]:
#load "Serverless.fsx"
open Avestura.Serverless
open MathNet.Numerics.Distributions

In [ ]:
// Visualization of the General.normalize
// function using Weilbul Stretched Function

[ for x in 0. .. 20 -> (x, General.normalize x) ]
|> Chart.Line

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
// Dijestra path finding

let dg = DependencyGraph.init (6u)

GraphExtensions.printAdjMatrix (dg.adjMatrix)

let path, distance = dg |> DependencyGraph.dijkestraPath (0u) (5u)
let weights = dg |> DependencyGraph.edgeWeightsFromPath path

path, weights, distance

--------------
|  559 | 885 | 689 | 210 | 704 | 0 |
--------------
|  455 | 6 | 717 | 21 | 635 | 634 |
--------------
|  286 | 0 | 0 | 225 | 302 | 0 |
--------------
|  150 | 537 | 0 | 0 | 0 | 288 |
--------------
|  0 | 99 | 0 | 0 | 854 | 636 |
--------------
|  12 | 0 | 90 | 100 | 0 | 739 |
--------------


Item1,Item2,Item3
"[ 0, 3, 5 ]","[ 210, 288 ]",498


In [ ]:
// fsevents
// hiredis
// karma-phantomjs-launcher
// phantomjs-prebuilt
// puppeteer
// serialport
// xml2json
// yeoman-environment
// let dd_impossible = [
//     "fsevents"
//     "hiredis"
//     "karma-phantomjs-launcher"
//     "phantomjs-prebuilt"
//     "puppeteer"
//     "serialport"
//     "xml2json"
//     "yeoman-environment"
// ]
// let dd_1 = PackagesData.dependentUpon.Keys |> List.ofSeq |> List.skipWhile(fun x -> x <> "yeoman-generator")
// PackagesData.downloadAndSaveAllPackageSizes dd_impossible

//dd_1.[1]

In [ ]:
let saturdayData = FrequenciesData.frequencies["saturday"]
Chart.Line(mapKeys saturdayData, mapValues saturdayData)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
open QueueDataGenerator
open QueueFunctionGeneration
open Simulator


let config1 : QueueFunctionGeneration.Options = 
    (IgnoreFrequencyData, IgnoreCoopNetwork, defaultUnawareRandomNormalDepsCount)
let queueData = generateFunctionQueueData config1 6
// let context = runSimulatonWithQueueData "saturday" queueData noMergeScheduler

let x = DependencyGraph(queueData.funcs)


Error: input.fsx (13,15)-(13,32) typecheck error This function takes too many arguments, or is used in a context where a function is not expected